In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE71220"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE71220"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE71220.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE71220.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE71220.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effect of statins on blood gene expression in COPD"
!Series_summary	"Background: COPD is currently the fourth leading cause of death worldwide and predicted to rank third by 2020. Statins are commonly used lipid lowering agents with documented benefits on cardiovascular morbidity and mortality, and have also been shown to have pleiotropic effects including anti-inflammatory and anti-oxidant activity.   Objective: Identify a gene signature associated with statin use in the blood of COPD patients, and identify molecular mechanisms and pathways underpinning this signature that could explain any potential benefits in COPD.  Methods: Whole blood gene expression was measured on 168 statin users and 452 non-users from the ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) study. Gene expression was measured using the Affymetrix  Human Gene 1.1 ST microarray chips. Factor Analysis for Robust Microarray Summarization

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, the dataset contains gene expression data measured using Affymetrix Human Gene 1.1 ST microarray chips
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# The background information indicates this is actually a COPD study, not a direct Huntington's Disease study
# While there's mention of gene signature correlation with Huntington's, there's no direct Huntington's Disease trait data
trait_row = None  # No direct Huntington's Disease information available

# Age information is available in row 2
age_row = 2

# Gender information is available in row 3 (labeled as 'Sex')
gender_row = 3

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert disease status to binary representation.
    However, in this dataset, there's no direct Huntington's Disease information.
    """
    return None  # No direct Huntington's Disease information

def convert_age(value):
    """Convert age to a continuous value."""
    if value is None or ":" not in value:
        return None
    
    age_str = value.split(": ")[1].strip()
    try:
        return float(age_str)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if value is None or ":" not in value:
        return None
    
    gender = value.split(": ")[1].strip()
    if gender.upper() == "F":
        return 0
    elif gender.upper() == "M":
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
# Since trait_row is None, we'll skip the clinical feature extraction step
print(f"No direct {trait} information available in this dataset.")
print(f"This is primarily a COPD study with some gene signature correlation to {trait} mentioned in the background.")


No direct Huntingtons_Disease information available in this dataset.
This is primarily a COPD study with some gene signature correlation to Huntingtons_Disease mentioned in the background.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE71220/GSE71220_series_matrix.txt.gz


Gene data shape: (12381, 617)
First 20 gene/probe identifiers:
Index(['7892501', '7892504', '7892507', '7892508', '7892509', '7892510',
       '7892514', '7892515', '7892516', '7892517', '7892520', '7892521',
       '7892527', '7892530', '7892531', '7892533', '7892534', '7892535',
       '7892536', '7892538'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown are numerical codes (like "7892501") that appear to be probe IDs
# rather than standard human gene symbols (which would be alphabetical like "GAPDH", "TP53", etc.)
# These are likely microarray probe identifiers that need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the preview, we see:
# - 'ID' column in gene_annotation contains probe IDs that match gene expression data index
# - 'gene_assignment' column contains gene symbols embedded in a complex string format

# 2. Create gene mapping dataframe
# Extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This function handles the many-to-many relationship as specified
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print gene data shape before and after mapping
print(f"Gene data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Gene data shape after mapping: (23114, 617)
First 10 gene symbols after mapping:
Index(['A-', 'A-52', 'A-I', 'A1', 'A1-', 'A10', 'A11', 'A12', 'A13', 'A14'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")
print("First 10 normalized gene symbols:")
print(gene_data_normalized.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# Since Step 2 determined that trait_row is None, this dataset doesn't contain
# relevant Huntington's Disease trait information
print("\nThis dataset does not contain relevant Huntington's Disease trait information.")
print("As determined in Step 2, this dataset is about COPD with only a mention of gene signature correlation to Huntington's Disease.")

# Create a minimal clinical data structure to document the absence of trait data
print("\nPreparing documentation of clinical data availability...")
empty_clinical_df = pd.DataFrame({trait: []})
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
empty_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data documentation saved to: {out_clinical_data_file}")

# No linking is possible since trait data is not available
print("\nSkipping data linking since trait information is not available.")

# Create a minimal dataframe with proper structure for validation
minimal_df = pd.DataFrame({trait: [0]})  # At least one row with trait column

# Conduct final quality validation - we already know this dataset is not usable for our specific trait
note = "This GSE71220 dataset contains gene expression data from COPD patients and mentions a correlation of gene signatures with Huntington's Disease in its description, but does not contain direct Huntington's Disease trait information."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=False,  # Set to False since trait unavailability is not a bias issue
    df=minimal_df,    # Provide minimally structured dataframe
    note=note
)

print(f"Dataset usability validation completed. Is usable: {is_usable}")
print("Dataset does not contain Huntington's Disease trait information, so linked data not created or saved.")

Normalizing gene symbols...
Gene data shape after normalization: (7901, 617)
First 10 normalized gene symbols:
Index(['A2M', 'A4GALT', 'AAA1', 'AAK1', 'AARS1', 'AASDH', 'AASDHPPT', 'AATF',
       'ABAT', 'ABCA1'],
      dtype='object', name='Gene')


Normalized gene data saved to: ../../output/preprocess/Huntingtons_Disease/gene_data/GSE71220.csv

This dataset does not contain relevant Huntington's Disease trait information.
As determined in Step 2, this dataset is about COPD with only a mention of gene signature correlation to Huntington's Disease.

Preparing documentation of clinical data availability...
Clinical data documentation saved to: ../../output/preprocess/Huntingtons_Disease/clinical_data/GSE71220.csv

Skipping data linking since trait information is not available.
Abnormality detected in the cohort: GSE71220. Preprocessing failed.
Dataset usability validation completed. Is usable: False
Dataset does not contain Huntington's Disease trait information, so linked data not created or saved.
